In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import string
import time

In [2]:
df = pd.read_csv('incheon.csv', index_col=0)

In [3]:
df

,인허가관할기관,업소명,업종,업태,업소주소,영업상태
0,인천광역시 강화군,(유한)애플이엔씨 인천강화,일반음식점,한식,인천광역시 강화군 선원면 창리 418 외 1필지,정상
1,인천광역시 강화군,(주)보광호,일반음식점,회집,인천광역시 강화군 길상면 해안남로619번길 21(1동 1층),정상
2,인천광역시 강화군,(주)섬고기,일반음식점,경양식,인천광역시 강화군 길상면 삼랑성길 4(1층),정상
3,인천광역시 강화군,(주)장어마을,일반음식점,한식,인천광역시 강화군 길상면 해안남로 298,정상
4,인천광역시 강화군,(주)충남서산집,일반음식점,한식,인천광역시 강화군 내가면 중앙로 1198(1층),정상
...,...,...,...,...,...,...
31201,인천광역시 서구,힘찬장어,일반음식점,한식,인천광역시 서구 중봉대로 800(1층 경서동),정상
31202,인천광역시 서구,힛트꼬치,일반음식점,호프/통닭,인천광역시 서구 원창로240번길 3(가정동),정상
31203,인천광역시 서구,힌무발,일반음식점,호프/통닭,인천광역시 서구 석남로 106(1층 일부 석남동),정상
31204,인천광역시 서구,돈까스 청라점,일반음식점,일식,"인천광역시 서구 중봉대로612번길 10-12(109,110호 연희동, 청라프라자2)",정상


In [13]:
# 데이터프레임 초기화
columns1 = ['id', 'title', 'category', 'address', 'tel', 'longitude', 'latitude', 'imgUrl' , 'summary', 'naverDataId']
columns2 = ['id', 'restaurantId', 'menuName', 'price', 'introduction', 'menuImgUrl']

df_restaurant = pd.DataFrame(columns=columns1)
df_menu = pd.DataFrame(columns=columns2)

In [96]:
# 인덱스 설정
index_df1 = 9000
index_df2 = 64768

In [15]:
#selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
driver = webdriver.Chrome("C:/Users/reodi/chromedriver_win32/chromedriver") 
driver.implicitly_wait(3) 

In [16]:
def naverMapCrawling(search):
    # row1 = ['id', 'title', 'category', 'address', 'tel', 'longitude,', 'latitude', 'imgUrl' , 'summary', 'naverDataId']
    # row2 = ['id', 'restaurantId', 'menuName', 'price', 'introduction', 'menuImgUrl']
    
    global index_df1
    global index_df2
    id = int(index_df1+1)
    
    # 검색
    driver.get(f"https://m.map.naver.com/search2/search.naver?query={search}") 
    time.sleep(1)
    items = driver.find_elements(By.CSS_SELECTOR, '._item ')
    
    # 검색결과가 없는 경우
    if not items:
        row1 = [id, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        df_restaurant.loc[index_df1] = row1
        index_df1 += 1
        return

    # 검색결과 중 첫번째를 저장
    title = items[0].get_attribute('data-title')
    category = items[0].find_element(By.TAG_NAME, 'em').text
    address = items[0].find_element(By.CSS_SELECTOR,'.item_address ').text.replace('주소보기\n', '')
    tel = items[0].get_attribute('data-tel')
    longitude = float(items[0].get_attribute('data-longitude'))
    latitude = float(items[0].get_attribute('data-latitude'))
    naverDataId = int(items[0].get_attribute('data-id'))
    
    for character in string.punctuation:
        title = title.replace(character, '') # 특수기호 제거
        
    # 홈탭 이동
    driver.get(f'https://m.place.naver.com/place/{naverDataId}/home')
    time.sleep(1)
    try:
        button = driver.find_element(By.CSS_SELECTOR, '.zPfVt')
        button.click()
        time.sleep(1)
        summary = driver.find_element(By.CSS_SELECTOR, '.zPfVt').text           
    except:
        summary = ''
    
    # 사진탭 이동
    driver.get(f'https://m.place.naver.com/place/{naverDataId}/photo')
    time.sleep(1)
    
    try:
        imgUrl = driver.find_element(By.ID, "업체사진_0").get_attribute('src')
    except:
        imgUrl = ''
    
    # df1에 행 추가
    row1 = [id, title, category, address, tel, longitude, latitude, imgUrl, summary, naverDataId]
    df_restaurant.loc[index_df1] = row1
    index_df1 += 1
    


    # 메뉴탭 이동
    driver.get(f'https://m.place.naver.com/restaurant/{naverDataId}/menu/list')
    time.sleep(1)
    
    # 일반 페이지인 경우
    menus = driver.find_elements(By.CLASS_NAME, 'P_Yxm')
    if menus:
        for menu in menus:
            name = menu.find_element(By.CLASS_NAME, 'Sqg65').text
            price = menu.find_element(By.CLASS_NAME, 'SSaNE').text
            introduction = menu.find_element(By.CLASS_NAME, 'eCaG_').text
            try:
                menuImg = menu.find_element(By.CLASS_NAME, 'K0PDV').get_attribute('style')
                menuImgUrl = menuImg.replace('width: 100px; height: 110px; background-image: url("', '').replace('");', '')
                menuImgUrl = menuImgUrl.replace('width: 100px; height: 100px; background-image: url("', '')
            except:
                menuImgUrl = ''
            
            row2 = [index_df2+1, id, name, price, introduction, menuImgUrl]
            df_menu.loc[index_df2] = row2
            index_df2 += 1       
    
    # 네이버 주문 페이지인 경우
    else:
        menus = driver.find_elements(By.CLASS_NAME, 'order_list_item')
        for menu in menus:
            name = menu.find_element(By.CLASS_NAME, 'tit').text
            price = menu.find_element(By.CLASS_NAME, 'price').text
            introduction = menu.find_element(By.CLASS_NAME, 'detail_txt').text
            try:
                menuImgUrl = menu.find_element(By.TAG_NAME, 'img').get_attribute('src')
            except:
                menuImgUrl = ''
                
            row2 = [index_df2+1, id, name, price, introduction, menuImgUrl]
            df_menu.loc[index_df2] = row2
            index_df2 += 1

In [17]:
search_list = df['인허가관할기관'] + ' ' + df['업소명']

In [89]:
search_list[17999:19000]

17999            인천광역시 남동구 최가네생고기
18000            인천광역시 남동구 최가네추어탕
18001           인천광역시 남동구 최가네포장마차
18002              인천광역시 남동구 최가박당
18003            인천광역시 남동구 최고당돈가스
                   ...           
18995         인천광역시 부평구 (진)순대국설렁탕
18996          인천광역시 부평구 (한결)통아구찜
18997    인천광역시 부평구 1.2.3~Z 베트남쌀국수
18998    인천광역시 부평구 101남산돈까스 롯데삼산점
18999          인천광역시 부평구 110(일일공)
Length: 1001, dtype: object

In [90]:
index_df1

8000

In [ ]:
# search_list 슬라이싱해서 돌리기
for search in search_list[19000:20000]:
    naverMapCrawling(search)
    if index_df1 % 100 == 0:
        print(index_df1-1)
        print(df_restaurant.loc[index_df1-1, 'title'])
        print('===============================')

In [92]:
df_restaurant

,id,title,category,address,tel,longitude,latitude,imgUrl,summary,naverDataId
0,1.0,화룡전설,중식당,인천광역시 미추홀구 경원대로864번길 39,032-422-4193,126.691637,37.460609,https://search.pstatic.net/common/?autoRotate=...,옆테이블이 보이지않는 칸막이가 있어 조용하고 아늑한 분위기입니다,5.476457e+07
1,2.0,화룽은중독,소고기구이,인천광역시 미추홀구 주안로104번길 57,032-435-8192,126.681365,37.461395,https://search.pstatic.net/common/?autoRotate=...,,1.646611e+07
2,3.0,화르화르 인천용현점,"치킨,닭강정",인천광역시 미추홀구 매소홀로 73-13 1층,032-881-8857,126.634180,37.449851,https://search.pstatic.net/common/?autoRotate=...,,3.666723e+07
3,4.0,화수분미용실,미용실,인천광역시 미추홀구 신기길15번길 5-1,,126.677026,37.447563,,어머니들이 많이 찾아가요,1.060150e+09
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8995,8996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8996,8997.0,한결통아구찜 산곡점,"아귀찜,해물찜",인천광역시 부평구 마장로 257-1 1층,032-277-7077,126.702513,37.501094,https://search.pstatic.net/common/?autoRotate=...,"아구찜중자가 19,900원인데작년과 같은액수로 거의모든 식당이 가격을 올렸는데 여긴...",1.456912e+09
8997,8998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8998,8999.0,101번지남산돈까스 롯데삼산점,101번지남산돈까스,인천광역시 부평구 길주로 623 롯데마트 삼산점 지하1층,032-512-2229,126.732063,37.508373,https://search.pstatic.net/common/?autoRotate=...,101번지 남산돈까스는 90년대 생긴 남산 최초의 한국식 수제왕돈까스 전문점으로 남...,1.068927e+09


In [93]:
df_menu

,id,restaurantId,menuName,price,introduction,menuImgUrl
0,1,1,삼선짬뽕,"8,500원",큰새우와 꽃게등 20여가지 재료와 진한 야채육수로 끓여낸 일품 삼선짬뽕입니다.,https://search.pstatic.net/common/?autoRotate=...
1,2,1,간짜장,"6,500원",물을 넣지 않고 순수한 양파로만 볶아낸 맛있는 짜장입니다. 짜장장을 따로 담지않고 ...,https://search.pstatic.net/common/?autoRotate=...
2,3,1,등심탕수육,"18,000원",,https://search.pstatic.net/common/?autoRotate=...
3,4,1,게살볶음밥,"8,500원",찐짜 겟살로 양상추와 여러가지 야채로 볶아낸 건강식 볶음밥입니다,https://search.pstatic.net/common/?autoRotate=...
4,5,2,눈꽃살,"23,000원",,
...,...,...,...,...,...,...
64763,64764,8999,냉모밀,"8,500원",탱탱한 메밀면에 깊은 맛의 쯔유가 어우러진 시원한 맛,https://search.pstatic.net/common/?autoRotate=...
64764,64765,8999,우동돈까스세트,"13,000원",따끈한 우동과 바삭한 돈까스의 환상적인 조화,https://search.pstatic.net/common/?autoRotate=...
64765,64766,8999,뚝배기김치우동세트,"13,000원",얼큰한 김치우동과 고소한 돈까스의 조화 *김치우동 + 1/2 남산왕돈까스,https://search.pstatic.net/common/?autoRotate=...
64766,64767,8999,쫄면돈까스세트,"13,000원",세상에 이런 만남은 없었다. 새콤달콤한 쫄면과 바삭한 돈까스! *쫄면 + 1/2돈까스,https://search.pstatic.net/common/?autoRotate=...


In [95]:
df_restaurant.to_csv('df_restaurant.csv')
df_menu.to_csv('df_menu.csv')

In [56]:
df.iloc[14893, :]

인허가관할기관                               인천광역시 남동구
업소명                                       둥지알까스
업종                                        일반음식점
업태                                          경양식
업소주소       인천광역시 남동구 간석로72번길 45-20(1층 101호 간석동)
영업상태                                         정상
Name: 14893, dtype: object